In [1]:
%pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.4 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=c5a5df0e0fc04b9e34ae57311b126eb6f8945b2982a04212c6396c7a8c4ddb59
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24570 sha256=8b69c9afbb4f56ee74079e207c8e68eab1e8c79d265a6df3475ca62bf09e613b
  Stored in directory: /root/.cache/pip/wheels/70/4b/f0/eaf5a8de646d8676dc25caa01949b9f9d883b8fa2efb435bc3
Successfully built bs4 parse


In [2]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [3]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [4]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set up your API key
api_key = 'YOUR API KEY'  # Replace with your actual API key

# Create a YouTube service object
youtube = build('youtube', 'v3', developerKey=api_key)

# Function to retrieve the YouTube account link from a video link
def get_account_link(video_url):
    try:
        if '?v=' in video_url:
            # Extract the video ID from the URL
            video_id = video_url.split('?v=')[1]

            # Call the API to get video details
            video_response = youtube.videos().list(
                part='snippet',
                id=video_id
            ).execute()

            # Extract the channel ID from the video details
            channel_id = video_response['items'][0]['snippet']['channelId']

            # Construct the YouTube account link
            account_link = f'https://www.youtube.com/channel/{channel_id}'

            return account_link
        elif "youtube.com/user/" in video_url or "youtube.com/channel/" in video_url or "youtube.com/c/" in video_url:
            return video_url
        else:
            return None

    except HttpError as e:
        print('An HTTP error occurred:')
        print(e)
        return None
    except KeyError:
        print('Invalid video URL. Please provide a valid YouTube video link.')
        return None

In [5]:
def scrape_google(query,num_links):
    query = urllib.parse.quote_plus(query)
    links = []
    account_links = []
    #num_links = 1000  # Number of links to retrieve
    num_results_per_page = 10  # Number of results per page

    while len(links) < num_links:
        page_num = len(links) // num_results_per_page + 1
        start_num = len(links) % num_results_per_page

        response = get_source("https://www.google.com/search?q={}&start={}".format(query, start_num))
        page_links = list(response.html.absolute_links)

        google_domains = (
            'https://www.google.',
            'https://google.',
            'https://webcache.googleusercontent.',
            'http://webcache.googleusercontent.',
            'https://policies.google.',
            'https://support.google.',
            'https://maps.google.'
        )

        for url in page_links[:]:
            if url.startswith(google_domains):
                page_links.remove(url)
            else:
                account_links.append(get_account_link(url))

        links.extend(page_links)

    return links[:num_links], account_links

In [6]:
scraped_links, scraped_account_links = scrape_google("site:youtube.com openinapp.co",1000)

In [7]:
print(scraped_links)

['https://www.youtube.com/watch?v=vOo-VyPIiW0', 'https://www.youtube.com/watch?v=s0eUGZVDdxU', 'https://www.youtube.com/watch?v=yRxXLMVd2es', 'https://www.youtube.com/watch?v=Njo_GPYbTvY', 'https://www.youtube.com/channel/UClOQrZIZQZojI3CmJ3jC2eg', 'https://www.youtube.com/watch?v=-GvsoRszRt4', 'https://www.youtube.com/watch?v=1IsfrcO1R-c', 'https://www.youtube.com/c/OpeninApp', 'https://www.youtube.com/watch?v=SHtfe6z7ekI', 'https://www.youtube.com/hashtag/openinapp', 'https://www.youtube.com/watch?v=vOo-VyPIiW0', 'https://www.youtube.com/watch?v=s0eUGZVDdxU', 'https://www.youtube.com/watch?v=yRxXLMVd2es', 'https://www.youtube.com/watch?v=Njo_GPYbTvY', 'https://www.youtube.com/channel/UClOQrZIZQZojI3CmJ3jC2eg', 'https://www.youtube.com/watch?v=-GvsoRszRt4', 'https://www.youtube.com/watch?v=1IsfrcO1R-c', 'https://www.youtube.com/c/OpeninApp', 'https://www.youtube.com/watch?v=SHtfe6z7ekI', 'https://www.youtube.com/hashtag/openinapp', 'https://www.youtube.com/watch?v=vOo-VyPIiW0', 'https

In [8]:
 # add the video links to a dataframe
df = pd.DataFrame(scraped_links, columns=['links'])

# export the dataframe to a csv file
df.to_csv('links.csv', index=False)

In [9]:
 # add the video links to a dataframe
df = pd.DataFrame(scraped_account_links, columns=['acc_links'])

# export the dataframe to a csv file
df.to_csv('acc_links.csv', index=False)